In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
from dateutil.relativedelta import relativedelta

credit = pd.read_csv("credit_record.csv").drop('Unnamed: 0', axis=1, errors='ignore')
credit['START_DATE'] = pd.to_datetime(credit['START_DATE'])
credit['SNAPSHOT_MONTH'] = pd.to_datetime(credit['SNAPSHOT_MONTH'])
credit.head()

,ID,STATUS,START_DATE,SNAPSHOT_MONTH
0,5001711,0,2022-09-01,2022-09-01
1,5001711,0,2022-09-01,2022-10-01
2,5001711,0,2022-09-01,2022-11-01
3,5001711,X,2022-09-01,2022-12-01
4,5001712,0,2021-06-01,2021-06-01


In [4]:
print("Unique values ==> ")
credit.apply(lambda x: x.nunique())

Unique values ==> 


ID                45971
STATUS                8
START_DATE           61
SNAPSHOT_MONTH       61
dtype: int64

In [5]:
print("Missing values values ==> ")
credit.apply(lambda x: x.isnull().sum())

Missing values values ==> 


ID                0
STATUS            0
START_DATE        0
SNAPSHOT_MONTH    0
dtype: int64

In [6]:
credit.STATUS.value_counts()

STATUS
C    441786
0    383019
X    209227
1     11059
5      1681
2       865
3       315
4       222
Name: count, dtype: int64

In [7]:
from operator import attrgetter


In [8]:
credit['n_period'] = (credit['SNAPSHOT_MONTH'].dt.to_period('M') - credit['START_DATE'].dt.to_period('M')).apply(attrgetter('n'))

In [9]:
credit['is_30_past_due'] = np.where(credit['STATUS'].isin(['1', '2', '3','4', '5']), 1, 0)

In [ ]:
# HAPUS C atau X jika perlu

In [11]:
cnt2 = credit.groupby(['START_DATE', 'n_period'])['is_30_past_due'].agg(['count', 'sum']).reset_index()

In [19]:
cnt2['cumsum'] = cnt2.sort_values(['START_DATE','n_period'],ascending=True).groupby(['START_DATE'])['sum'].cumsum()

In [21]:
cnt2['bad'] = cnt2['cumsum']/cnt2['count']

In [22]:
cnt2

,START_DATE,n_period,count,sum,cumsum,bad
0,2017-12-01,0,415,0,0,0.000000
1,2017-12-01,1,414,2,2,0.004831
2,2017-12-01,2,410,6,8,0.019512
3,2017-12-01,3,408,5,13,0.031863
4,2017-12-01,4,403,9,22,0.054591
...,...,...,...,...,...,...
1886,2022-10-01,1,770,14,14,0.018182
1887,2022-10-01,2,750,14,28,0.037333
1888,2022-11-01,0,684,0,0,0.000000
1889,2022-11-01,1,680,10,10,0.014706
